在机器学习中，离散化（Discretization） 是将连续数值特征转换为离散类别特征的过程，而二值化（Binarization） 则是将数值特征转换为 0/1 二值的特殊情况。这两种技术常用于简化模型、增强鲁棒性或处理特定类型的算法（如规则树模型）。下面详细介绍 KBinsDiscretizer 和 Binarizer 的用法、参数及案例。

## 一、KBinsDiscretizer：分箱离散化
1. 核心作用
    将连续数值特征划分为多个离散区间（分箱），每个区间作为一个类别。例如，将年龄分为 “青年”“中年”“老年”。
2. 关键参数

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(
    n_bins=5,                # 每个特征的分箱数量
    encode='onehot',         # 编码方式：'onehot'、'onehot-dense'、'ordinal'
    strategy='quantile',     # 分箱策略：'uniform'、'quantile'、'kmeans'
    dtype=np.float64,        # 输出数据类型
    subsample=1e5,           # 拟合时的最大样本数（内存优化）
    random_state=None        # 随机种子（仅用于kmeans策略）
)

3. 参数详解
* n_bins：
    每个特征的分箱数量，可以是整数（所有特征相同）或数组（为每个特征指定不同数量）。
* encode：
    - 'onehot'（默认）：返回稀疏的独热编码矩阵；
    - 'onehot-dense'：返回密集的独热编码矩阵；
    - 'ordinal'：返回有序整数编码（如 0,1,2...）。
* strategy：
    - 'uniform'：等宽分箱，每个区间宽度相等；
    - 'quantile'：等频分箱，每个区间包含近似相同数量的样本；
    - 'kmeans'：基于 K-means 聚类的分箱，区间边界由聚类中心决定。

In [1]:
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer

# 创建示例数据（年龄和收入）
X = np.array([
    [20, 5000],
    [30, 6000],
    [40, 7000],
    [50, 8000],
    [60, 9000]
])

# 创建离散化器（等频分箱，独热编码）
discretizer = KBinsDiscretizer(
    n_bins=[2, 3],          # 年龄分2箱，收入分3箱
    encode='onehot-dense',  # 返回密集矩阵
    strategy='quantile'     # 等频分箱
)

In [2]:
# 拟合并转换数据
X_discretized = discretizer.fit_transform(X)

print("原始数据：")
print(X)
print("\n离散化后的数据：")
print(X_discretized)  # 输出：[[1. 0. 1. 0. 0.], [1. 0. 0. 1. 0.], ...]

原始数据：
[[  20 5000]
 [  30 6000]
 [  40 7000]
 [  50 8000]
 [  60 9000]]

离散化后的数据：
[[1. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]]


In [3]:
# 查看分箱边界
print("\n分箱边界：")
for feature_idx, bins in enumerate(discretizer.bin_edges_):
    print(f"特征 {feature_idx} 的分箱边界：{bins}")
# 输出：特征0(年龄)边界：[20. 45. 60.]，特征1(收入)边界：[5000. 6500. 8500. 9000.]


分箱边界：
特征 0 的分箱边界：[20. 40. 60.]
特征 1 的分箱边界：[5000.         6333.33333333 7666.66666667 9000.        ]


In [4]:
# 使用有序编码
discretizer_ordinal = KBinsDiscretizer(
    n_bins=2,
    encode='ordinal',
    strategy='uniform'  # 等宽分箱
)
X_ordinal = discretizer_ordinal.fit_transform(X)
print("\n有序编码结果：")
print(X_ordinal)  # 输出：[[0. 0.], [0. 0.], [1. 1.], [1. 1.], [1. 1.]]


有序编码结果：
[[0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
